In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.etree as etree
import datetime
import time
import re

In [2]:
df = pd.DataFrame()

In [3]:
df.at['ding', 'bing'] = 10

In [4]:
df

,bing
ding,10.0


In [2]:
AGENT = 'http://localhost:5000/'
CURRENT = 'current'
PROBE = 'probe'
CATEGORIES = ['Samples', 'Events', "Condition"]

In [3]:
resp = requests.get('http://localhost:5000/current')
root = etree.fromstring(resp.content)

In [4]:
NAMESPACE = re.split('{|}', root.tag)[1]
NAMESPACE

'urn:mtconnect.org:MTConnectStreams:1.5'

In [8]:
device = root.xpath("//x:DeviceStream[@uuid='S202100470FKBAH']", namespaces={'x': NAMESPACE})
device

[<Element {urn:mtconnect.org:MTConnectStreams:1.5}DeviceStream at 0x222a51e37c0>]

In [19]:
for ting in root.xpath("//x:" + CATEGORIES[2], namespaces={'x': NAMESPACE}):
    for data in ting:
        print(data.attrib)
        print(data.text)

{'dataItemId': 'Cutting_condition', 'nativeCode': 'INACTIVE', 'nativeSeverity': '0', 'qualifier': 'NORMAL', 'sequence': '109', 'timestamp': '2023-11-01T02:14:22.839145Z', 'type': 'SYSTEM'}
NO CUTTING


In [11]:
root.tag

'{urn:mtconnect.org:MTConnectStreams:1.5}MTConnectStreams'

In [13]:
device = 'S202100470FKBAH'
root.xpath("//x:DeviceStream[@uuid='" + device + "']", namespaces={'x': NAMESPACE})

[<Element {urn:mtconnect.org:MTConnectStreams:1.5}DeviceStream at 0x222a51e37c0>]

In [25]:
newdf = pd.DataFrame()

[[[newdf.at[pd.to_datetime(seq.attrib['timestamp']).round('ms'), seq.tag] = seq.text for seq in data] for data in root.xpath("//x:" + category, namespaces={'x': NAMESPACE})] for category in CATEGORIES]

SyntaxError: cannot assign to subscript here. Maybe you meant '==' instead of '='? (1738645675.py, line 3)

In [26]:
for category in CATEGORIES:
    for data in root.xpath("//x:" + category, namespaces={'x': NAMESPACE}):
        for seq in data:
            newdf.at[pd.to_datetime(seq.attrib['timestamp']).round('ms'), seq.tag] = seq.text

In [27]:
newdf

,{urn:mtconnect.org:MTConnectStreams:1.5}AccumulatedTime,{urn:mtconnect.org:MTConnectStreams:1.5}PathFeedrateOverride,{urn:mtconnect.org:MTConnectStreams:1.5}PathFeedrate,{urn:mtconnect.org:MTConnectStreams:1.5}RapidFeedrate,{urn:mtconnect.org:MTConnectStreams:1.5}RotaryVelocity,{urn:mtconnect.org:MTConnectStreams:1.5}RotaryVelocityOverride,{urn:mtconnect.org:MTConnectStreams:1.5}LinearVelocity,{urn:mtconnect.org:MTConnectStreams:1.5}XAxisPosition,{urn:mtconnect.org:MTConnectStreams:1.5}YAxisPosition,{urn:mtconnect.org:MTConnectStreams:1.5}ZAxisPosition,...,{urn:mtconnect.org:MTConnectStreams:1.5}Availability,{urn:mtconnect.org:MTConnectStreams:1.5}AssetChanged,{urn:mtconnect.org:MTConnectStreams:1.5}AssetRemoved,{urn:mtconnect.org:MTConnectStreams:1.5}Line,{urn:mtconnect.org:MTConnectStreams:1.5}ToolNumber,{urn:mtconnect.org:MTConnectStreams:1.5}ProgramStatus,{urn:mtconnect.org:MTConnectStreams:1.5}PartCount,{urn:mtconnect.org:MTConnectStreams:1.5}ProgramEditName,{urn:mtconnect.org:MTConnectStreams:1.5}Program,{urn:mtconnect.org:MTConnectStreams:1.5}Normal
2023-11-01 02:14:09.491000+00:00,UNAVAILABLE,NaN,UNAVAILABLE,UNAVAILABLE,NaN,NaN,NaN,NaN,NaN,NaN,...,UNAVAILABLE,UNAVAILABLE,UNAVAILABLE,UNAVAILABLE,NaN,NaN,UNAVAILABLE,NaN,NaN,NaN
2023-11-01 02:14:09.490000+00:00,UNAVAILABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:22.838000+00:00,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19,STOPPED,NaN,PBALL6061.NC,PBALL6061.NC,NaN
2023-11-01 02:14:22.837000+00:00,NaN,NaN,NaN,NaN,0,100,7,NaN,NaN,NaN,...,AVAILABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:27.027000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311.440092775002,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:27.274000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.840823707969,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:27.029000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-30.3197673494414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:22.839000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO CUTTING


In [30]:
df = pd.DataFrame()
df

""


In [34]:
pd.concat([df, newdf])

,{urn:mtconnect.org:MTConnectStreams:1.5}AccumulatedTime,{urn:mtconnect.org:MTConnectStreams:1.5}PathFeedrateOverride,{urn:mtconnect.org:MTConnectStreams:1.5}PathFeedrate,{urn:mtconnect.org:MTConnectStreams:1.5}RapidFeedrate,{urn:mtconnect.org:MTConnectStreams:1.5}RotaryVelocity,{urn:mtconnect.org:MTConnectStreams:1.5}RotaryVelocityOverride,{urn:mtconnect.org:MTConnectStreams:1.5}LinearVelocity,{urn:mtconnect.org:MTConnectStreams:1.5}XAxisPosition,{urn:mtconnect.org:MTConnectStreams:1.5}YAxisPosition,{urn:mtconnect.org:MTConnectStreams:1.5}ZAxisPosition,...,{urn:mtconnect.org:MTConnectStreams:1.5}Availability,{urn:mtconnect.org:MTConnectStreams:1.5}AssetChanged,{urn:mtconnect.org:MTConnectStreams:1.5}AssetRemoved,{urn:mtconnect.org:MTConnectStreams:1.5}Line,{urn:mtconnect.org:MTConnectStreams:1.5}ToolNumber,{urn:mtconnect.org:MTConnectStreams:1.5}ProgramStatus,{urn:mtconnect.org:MTConnectStreams:1.5}PartCount,{urn:mtconnect.org:MTConnectStreams:1.5}ProgramEditName,{urn:mtconnect.org:MTConnectStreams:1.5}Program,{urn:mtconnect.org:MTConnectStreams:1.5}Normal
2023-11-01 02:14:09.491000+00:00,UNAVAILABLE,NaN,UNAVAILABLE,UNAVAILABLE,NaN,NaN,NaN,NaN,NaN,NaN,...,UNAVAILABLE,UNAVAILABLE,UNAVAILABLE,UNAVAILABLE,NaN,NaN,UNAVAILABLE,NaN,NaN,NaN
2023-11-01 02:14:09.490000+00:00,UNAVAILABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:22.838000+00:00,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19,STOPPED,NaN,PBALL6061.NC,PBALL6061.NC,NaN
2023-11-01 02:14:22.837000+00:00,NaN,NaN,NaN,NaN,0,100,7,NaN,NaN,NaN,...,AVAILABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:27.027000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311.440092775002,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:27.274000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.840823707969,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:27.029000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-30.3197673494414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:22.839000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO CUTTING


In [40]:
df1 = pd.DataFrame()
df1.at[0, 'a'] = 20
df1.at[1, 'a'] = 10
df2 = pd.DataFrame()
df2.at[0, ('a', 'b')] = (4, 15)
pd.concat([df1, df2])

,a,b
0,20.0,NaN
1,10.0,NaN
0,4.0,15.0


In [79]:
header = root.xpath("//x:Header", namespaces={'x': NAMESPACE})[0].attrib
header

{'creationTime': '2023-11-01T02:14:27Z', 'sender': 'AcerSwiftX', 'instanceId': '1698804849', 'version': '1.7.0.2', 'bufferSize': '32768', 'nextSequence': '144', 'firstSequence': '1', 'lastSequence': '143'}

In [81]:
import math

sequence = 143

testdf = pd.DataFrame(columns=['timestamp'])
for category in CATEGORIES:
    for data in root.xpath("//x:" + category, namespaces={'x': NAMESPACE}):
        for seq in data:
            testdf.at[0, seq.tag] = seq.text
            if int(seq.attrib['sequence']) == sequence:
                testdf.at[0, 'timestamp'] = pd.to_datetime(seq.attrib['timestamp']).round('ms')
            
testdf

,timestamp,{urn:mtconnect.org:MTConnectStreams:1.5}AccumulatedTime,{urn:mtconnect.org:MTConnectStreams:1.5}PathFeedrateOverride,{urn:mtconnect.org:MTConnectStreams:1.5}PathFeedrate,{urn:mtconnect.org:MTConnectStreams:1.5}RapidFeedrate,{urn:mtconnect.org:MTConnectStreams:1.5}RotaryVelocity,{urn:mtconnect.org:MTConnectStreams:1.5}RotaryVelocityOverride,{urn:mtconnect.org:MTConnectStreams:1.5}LinearVelocity,{urn:mtconnect.org:MTConnectStreams:1.5}XAxisPosition,{urn:mtconnect.org:MTConnectStreams:1.5}YAxisPosition,...,{urn:mtconnect.org:MTConnectStreams:1.5}Availability,{urn:mtconnect.org:MTConnectStreams:1.5}AssetChanged,{urn:mtconnect.org:MTConnectStreams:1.5}AssetRemoved,{urn:mtconnect.org:MTConnectStreams:1.5}Line,{urn:mtconnect.org:MTConnectStreams:1.5}ToolNumber,{urn:mtconnect.org:MTConnectStreams:1.5}ProgramStatus,{urn:mtconnect.org:MTConnectStreams:1.5}PartCount,{urn:mtconnect.org:MTConnectStreams:1.5}ProgramEditName,{urn:mtconnect.org:MTConnectStreams:1.5}Program,{urn:mtconnect.org:MTConnectStreams:1.5}Normal
0,2023-11-01 02:14:27.274000+00:00,UNAVAILABLE,20,UNAVAILABLE,UNAVAILABLE,0,100,7,311.440092775002,293.840823707969,...,UNAVAILABLE,UNAVAILABLE,UNAVAILABLE,UNAVAILABLE,19,STOPPED,UNAVAILABLE,PBALL6061.NC,PBALL6061.NC,NO CUTTING


In [50]:
def record(df: pd.DataFrame, content: str, device: str, namespace: str) -> None:
    '''Record sensor data from XML string for given sensor.
    
    This function converts `content` into an ElementTree object and then
    uses XPath to find the data for the given sensor and inserts the data
    into the given data frame.
    '''
    
    root = etree.fromstring(content)
    root = root.xpath("//x:DeviceStream[@uuid='" + device + "']", namespaces={'x': namespace})[0]
    
    new_df = pd.DataFrame()
    for category in CATEGORIES:
        for data in root.xpath("//x:" + category, namespaces={'x': namespace}):
            for seq in data:
                new_df.at[pd.to_datetime(seq.attrib['timestamp']).round('ms'), seq.tag] = seq.text

    # lock.acquire()
    return pd.concat([df, new_df])
    # lock.release()

In [51]:
df = pd.DataFrame()
record(df, resp.content, 'S202100470FKBAH', NAMESPACE)

,{urn:mtconnect.org:MTConnectStreams:1.5}AccumulatedTime,{urn:mtconnect.org:MTConnectStreams:1.5}PathFeedrateOverride,{urn:mtconnect.org:MTConnectStreams:1.5}PathFeedrate,{urn:mtconnect.org:MTConnectStreams:1.5}RapidFeedrate,{urn:mtconnect.org:MTConnectStreams:1.5}RotaryVelocity,{urn:mtconnect.org:MTConnectStreams:1.5}RotaryVelocityOverride,{urn:mtconnect.org:MTConnectStreams:1.5}LinearVelocity,{urn:mtconnect.org:MTConnectStreams:1.5}XAxisPosition,{urn:mtconnect.org:MTConnectStreams:1.5}YAxisPosition,{urn:mtconnect.org:MTConnectStreams:1.5}ZAxisPosition,...,{urn:mtconnect.org:MTConnectStreams:1.5}Availability,{urn:mtconnect.org:MTConnectStreams:1.5}AssetChanged,{urn:mtconnect.org:MTConnectStreams:1.5}AssetRemoved,{urn:mtconnect.org:MTConnectStreams:1.5}Line,{urn:mtconnect.org:MTConnectStreams:1.5}ToolNumber,{urn:mtconnect.org:MTConnectStreams:1.5}ProgramStatus,{urn:mtconnect.org:MTConnectStreams:1.5}PartCount,{urn:mtconnect.org:MTConnectStreams:1.5}ProgramEditName,{urn:mtconnect.org:MTConnectStreams:1.5}Program,{urn:mtconnect.org:MTConnectStreams:1.5}Normal
2023-11-01 02:14:09.491000+00:00,UNAVAILABLE,NaN,UNAVAILABLE,UNAVAILABLE,NaN,NaN,NaN,NaN,NaN,NaN,...,UNAVAILABLE,UNAVAILABLE,UNAVAILABLE,UNAVAILABLE,NaN,NaN,UNAVAILABLE,NaN,NaN,NaN
2023-11-01 02:14:09.490000+00:00,UNAVAILABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:22.838000+00:00,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19,STOPPED,NaN,PBALL6061.NC,PBALL6061.NC,NaN
2023-11-01 02:14:22.837000+00:00,NaN,NaN,NaN,NaN,0,100,7,NaN,NaN,NaN,...,AVAILABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:27.027000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311.440092775002,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:27.274000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.840823707969,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:27.029000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-30.3197673494414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01 02:14:22.839000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO CUTTING


In [12]:
timer = 5

start = 1
count = 100
url_f = "http://localhost:5000/sample?from={}&count={}".format(start, count)
url_c = "http://localhost:5000/current"
print(url_f)

resp = requests.get(url_c)
root = etree.fromstring(resp.content)
pd.to_datetime(datetime.datetime.utcnow())

http://localhost:5000/sample?from=1&count=1000


Timestamp('2023-10-18 17:13:30.585739')

In [24]:
root[0].attrib

{'creationTime': '2023-10-18T17:13:30Z', 'sender': 'AcerSwiftX', 'instanceId': '1697649171', 'version': '1.7.0.2', 'bufferSize': '32768', 'nextSequence': '221', 'firstSequence': '1', 'lastSequence': '220'}

In [19]:
pd.to_datetime(root[0].attrib['creationTime'])

Timestamp('2023-10-18 17:13:30+0000', tz='UTC')

In [42]:
# Create an empty DataFrame with row indices starting from 10000
index_range = range(221)

# Create the DataFrame
df = pd.DataFrame(index=index_range, columns=['timestamp'])

for j in root[1]:
    for k in j:
        for l in k:
            for m in l:
                if m.attrib['sequence'] == '220': print(pd.to_datetime(m.attrib['timestamp']))

2023-10-18 17:13:30.459350+00:00


In [63]:
a = pd.to_datetime(datetime.datetime.utcnow())
b = 0
delta = 0
while (delta < 2) :
    b = pd.to_datetime(datetime.datetime.utcnow())
    delta = (b - a).seconds

print(delta, b, a)

2 2023-10-18 17:30:18.415404 2023-10-18 17:30:16.415139


In [86]:
timer = 2
url_c = "http://localhost:5000/current"

# Get current data and latest stream
resp = requests.get(url_c)
root1 = etree.fromstring(resp.content)

# Get current time
t = pd.to_datetime(datetime.datetime.utcnow())

time.sleep(timer)



start = root[0].attrib['nextSequence']
count = 1
url_f = "http://localhost:5000/sample?from={}&count={}".format(start, count)

start 6859
http://localhost:5000/sample?from=6859&count=1


In [110]:
df = pd.DataFrame(columns=['num'])

In [111]:
df.at[2, 'num'] = 2

In [112]:
df

,num
2,2


In [113]:
df

,num
2,2


In [114]:
df.at[3, 'num'] = 4

In [115]:
df.at[4, 'num2'] = 3

In [116]:
df

,num,num2
2,2,NaN
3,4,NaN
4,NaN,3.0


In [131]:
df.at[3, 'num']

4

In [128]:
df.loc[3:]

,num,num2
3,4,NaN
4,NaN,3.0


In [103]:
url_f = "http://localhost:5000/sample?from={}&count={}".format(10, 1)
resp = requests.get(url_f)
root = etree.fromstring(resp.content)

In [107]:
root[1][0][0][0][0].attrib['timestamp']

'2023-10-18T18:23:51.201447Z'

In [28]:
import keyboard

print("Press any key to continue...")

# Wait for a keypress
keyboard.read_event(suppress=True)  # This line blocks until a key is pressed

# Print something after a keypress
print("A key was pressed!")

Press any key to continue...
A key was pressed!


In [10]:
time = pd.to_datetime(datetime.datetime.utcnow())
time

Timestamp('2023-10-22 18:33:44.149979')

In [4]:
time.round('ms').to_datetime64()

numpy.datetime64('2023-10-22T18:33:30.963000000')

In [5]:
temp  = pd.DataFrame(columns=['timestamp', 'value'])

In [6]:
# set timestamp column as index

temp.set_index('timestamp', inplace=True)
temp

,value
timestamp,


In [11]:
temp.at[time.round('ms'), 'value'] = 4
temp

,value
timestamp,
2023-10-22 17:00:15.229,9
2023-10-22 18:33:30.963,4
2023-10-22 18:33:44.150,4


In [15]:
newtime = pd.to_datetime('2023-10-22T17:00:15.229000000')

temp.at[newtime.round('ms'), 'value'] = 0
temp

,value
timestamp,
2023-10-22 17:00:15.229,9
2023-10-22 18:33:30.963,4
2023-10-22 18:33:44.150,4
2023-10-22 17:00:14.229,0


In [17]:
temp.sort_index(inplace=True)
temp

,value
timestamp,
2023-10-22 17:00:14.229,0
2023-10-22 17:00:15.229,9
2023-10-22 18:33:30.963,4
2023-10-22 18:33:44.150,4


In [21]:
temp.loc[pd.to_datetime('2023-10-22T17:00:16.229000000'): ]

,value
timestamp,
2023-10-22 18:33:30.963,4
2023-10-22 18:33:44.150,4


In [61]:
temp2 = temp
temp2.reset_index(inplace=True)

In [62]:
temp.index

RangeIndex(start=0, stop=4, step=1)

In [65]:
temp

,timestamp,value
0,2023-10-22 17:00:14.229,-1
1,2023-10-22 17:00:15.229,9
2,2023-10-22 18:33:30.963,4
3,2023-10-22 18:33:44.150,4


In [49]:
temp.loc[1, ('value')]

KeyError: 1

In [42]:
temp['value'][0] = -1

In [43]:
temp

,value
timestamp,
2023-10-22 17:00:14.229,-1
2023-10-22 17:00:15.229,9
2023-10-22 18:33:30.963,4
2023-10-22 18:33:44.150,4


In [27]:
print(pd.to_datetime(datetime.datetime.utcnow()))
resp = requests.get('http://localhost:5000/current')
print(pd.to_datetime(datetime.datetime.utcnow()))

2023-10-22 18:41:05.364936
2023-10-22 18:41:07.422499


In [38]:
root = etree.fromstring(resp.content)
start = root[0].attrib['nextSequence']
start

'368'

In [39]:
resp2 = requests.get('http://localhost:5000/sample?from={}&count={}'.format(start, 1))
root2 = etree.fromstring(resp2.content)
root2[1][0][0][0][0].attrib['timestamp']

'2023-10-22T18:41:07.466314Z'

In [80]:
temp.iloc[2, 'value']

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [79]:
temp

,value
timestamp,
2023-10-22 17:00:14.229,-1
2023-10-22 17:00:15.229,9
2023-10-22 18:33:30.963,100
2023-10-22 18:33:44.150,4


In [81]:
temp.reset_index(inplace=True, names='yeet')

In [82]:
temp

,yeet,value
0,2023-10-22 17:00:14.229,-1
1,2023-10-22 17:00:15.229,9
2,2023-10-22 18:33:30.963,100
3,2023-10-22 18:33:44.150,4


In [93]:
import time
time.tzname

('US Eastern Standard Time', 'US Eastern Daylight Time')

In [97]:
# pd.to_datetime(datetime.datetime.utcnow()).tz_localize(time.tzname[0])

TypeError: descriptor 'tzname' for 'datetime.datetime' objects doesn't apply to a 'str' object